<a href="https://www.kaggle.com/code/mikemollel/swahili-gemma-inference-with-inference-with-huggin?scriptVersionId=170077887" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 14.9 MB/s eta 0:00:00


In [2]:
# !pip install triton

In [3]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model = AutoPeftModelForCausalLM.from_pretrained(
    "Mollel/Gemma_Swahili_Mollel_1_epoch",
    load_in_4bit = False
)

tokenizer = AutoTokenizer.from_pretrained("Mollel/Gemma_Swahili_Mollel_1_epoch")

adapter_config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]

In [4]:
input_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [5]:
input_text = input_prompt.format(
        "Andika aya fupi kuhusu mada iliyotolewa.", # instruction
        "Umuhimu wa kutumia nishati inayoweza kurejeshwa", # input
        "", # output - leave this blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
response = tokenizer.batch_decode(outputs)[0]

2024-04-03 11:04:58.105730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 11:04:58.105828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 11:04:58.469669: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
response

'<bos>\n### Instruction:\nAndika aya fupi kuhusu mada iliyotolewa.\n\n### Input:\nUmuhimu wa kutumia nishati inayoweza kurejeshwa\n\n### Response:\nUmuhimu wa kutumia nishati mbadala ni muhimu kwa ajili ya kuhifadhi mazingira yetu na kuhakikisha usalama wa muda mrefu wa sayari yetu. Vyanzo vya nishati mbadala kama vile jua, upepo, na umeme wa maji hutoa uzalishaji mdogo au hakuna uzalishaji wa kaboni, na kupunguza alama yetu ya kaboni na kupunguza uchafuzi wa hewa. Kwa kuongeza, nishati mbadala ni rasilimali mbadala, tofauti na mafuta ya makaa ya mawe ambayo ni ya mwisho. Kwa kugeuza kwa nishati mbadala, tunaweza kupunguza utegemezi wetu kwa vyanzo visivyoweza kubadilika na kuunda siku zijazo endelevu zaidi kwa sayari yetu.<eos>'

**AI2 Reasoning Challenge**

In [7]:
input_text = input_prompt.format(
        "Ni kigezo kipi kinachoweza kusababisha mtu kupata homa kwa uwezekano mkubwa zaidi? Kila chaguo la maandishi lina lebo inayolingana nayo (kwa mfano, 'idadi ya bakteria katika damu' lina lebo ya 'B'). Chagua lebo sahihi inayoendana na jibu.", # instruction
        "(A) misuli ya mguu kupumzika baada ya mazoezi, (B) idadi ya bakteria katika damu, (C) vijidudu vya virusi kwenye ngozi, (D) wanga kumeng'enywa tumboni", # input
        "", # output - leave this blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)

response = tokenizer.batch_decode(outputs)

In [8]:
response

["<bos>\n### Instruction:\nNi kigezo kipi kinachoweza kusababisha mtu kupata homa kwa uwezekano mkubwa zaidi? Kila chaguo la maandishi lina lebo inayolingana nayo (kwa mfano, 'idadi ya bakteria katika damu' lina lebo ya 'B'). Chagua lebo sahihi inayoendana na jibu.\n\n### Input:\n(A) misuli ya mguu kupumzika baada ya mazoezi, (B) idadi ya bakteria katika damu, (C) vijidudu vya virusi kwenye ngozi, (D) wanga kumeng'enywa tumboni\n\n### Response:\n(B) idadi ya bakteria katika damu<eos>"]